In [ ]:
%matplotlib inline
from collections import OrderedDict

import numpy as np
import xarray as xr

from pumahu.syn_model import (SynModel,
                              setup_test, setup_realistic,
                              resample)

from pumahu.visualise import trellis_plot
from pumahu.mcmc import ns_sampling
from pumahu.uks import UnscentedKalmanSmoother
from pumahu import get_data

## Simple synthetic model

In [ ]:
xds1 = SynModel().run(setup_test(), gradient=True)
xds2 = SynModel(integration_method='rk4').run(setup_test(), gradient=True)
trellis_plot(xds1, xds2)

### Invert the model

In [ ]:
xds1_inv = ns_sampling(xds1.exp, mass2area=SynModel().mass2area)

In [ ]:
trellis_plot(xds1_inv, xds1)

In [ ]:
uks = UnscentedKalmanSmoother(data=xds1.exp)
xds1_uks = uks(test=True)

In [ ]:
trellis_plot(xds1_uks, xds1_inv)

## A more complex synthetic model

In [ ]:
xds3 = SynModel().run(setup_realistic(sinterval=24*60))
trellis_plot(xds3)

In [ ]:
# Uncomment if you have time
# xds3_inv = ns_sampling(xds3.exp, mass2area=SynModel().mass2area)

In [ ]:
uks = UnscentedKalmanSmoother(data=xds3.exp)
xds3_uks = uks(test=True, smooth=True)

In [ ]:
trellis_plot(xds3_uks, xds3, filled_error=True)

### Now we add some noise

In [ ]:
xds4 = SynModel().run(setup_realistic(sinterval=120), addnoise=True)

In [ ]:
trellis_plot(xds4)

### Resample the data as preparation for the inversion

In [ ]:
na = resample(xds4)
trellis_plot(xds4, xr.Dataset({'exp': na}))

### Invert the resampled data

In [ ]:
# Uncomment if you have time
# xds4_inv = ns_sampling(na, mass2area=SynModel().mass2area)

In [ ]:
uks4 = UnscentedKalmanSmoother(data=xds4.exp)
xds4_uks = uks4(test=True, smooth=True, alpha=0.35)

In [ ]:
trellis_plot(xds4_uks, filled_error=False)